# Domain Layer Refactoring Plan

This notebook guides the refactoring of Domain Aggregates, Events, and Application Handlers for the following modules:
1. Permissions (權限)
2. Documents (文件)
3. Tasks (任務)
4. Daily (每日紀錄)

## Objectives
- Fix `max-params` lint errors.
- Align code with documentation `docs/modulars/`.
- Use Parameter Object pattern for Event Creators.

## Steps
1.  **Read Documentation**: Read the modular documentation files.
2.  **Define Interfaces**: Define `params` interfaces for events.
3.  **Refactor Events**: Update Event Creator functions.
4.  **Update Handlers**: Update Application Handlers.
5.  **Verify Aggregates**: Ensure alignment with docs.

In [ ]:
import re

task_status_path = r'src/app/modules/tasks/domain/value-objects/task-status.vo.ts'

with open(task_status_path, 'r', encoding='utf-8') as f:
    content = f.read()

new_content = """/**
 * TaskStatus Value Object
 * 
 * Layer: Domain
 * DDD Pattern: Value Object
 */
export enum TaskStatusEnum {
    TODO = 'TODO',
    IN_PROGRESS = 'IN_PROGRESS',
    IN_QC = 'IN_QC',
    QC_FAILED = 'QC_FAILED',
    IN_ACCEPTANCE = 'IN_ACCEPTANCE',
    ACCEPTED = 'ACCEPTED',
    COMPLETED = 'COMPLETED',
    BLOCKED = 'BLOCKED',
    IN_REVIEW = 'IN_REVIEW',
    REJECTED = 'REJECTED',
    DRAFT = 'DRAFT',
    READY = 'READY'
}

export class TaskStatus {
    private readonly value: TaskStatusEnum;
    private static readonly cache = new Map<string, TaskStatus>();

    private constructor(value: TaskStatusEnum) {
        this.value = value;
    }

    public static create(value: string | TaskStatusEnum): TaskStatus {
        const strValue = value as string;
        if (TaskStatus.cache.has(strValue)) {
            return TaskStatus.cache.get(strValue)!;
        }

        const validValues = Object.values(TaskStatusEnum) as string[];
        if (validValues.includes(strValue)) {
            const instance = new TaskStatus(strValue as TaskStatusEnum);
            TaskStatus.cache.set(strValue, instance);
            return instance;
        }
        throw new Error(`Invalid TaskStatus: ${value}`);
    }

    // Static Instances for Policy/Enum-like usage
    public static readonly TODO = TaskStatus.create(TaskStatusEnum.TODO);
    public static readonly IN_PROGRESS = TaskStatus.create(TaskStatusEnum.IN_PROGRESS);
    public static readonly IN_QC = TaskStatus.create(TaskStatusEnum.IN_QC);
    public static readonly QC_FAILED = TaskStatus.create(TaskStatusEnum.QC_FAILED);
    public static readonly IN_ACCEPTANCE = TaskStatus.create(TaskStatusEnum.IN_ACCEPTANCE);
    public static readonly ACCEPTED = TaskStatus.create(TaskStatusEnum.ACCEPTED);
    public static readonly COMPLETED = TaskStatus.create(TaskStatusEnum.COMPLETED);
    public static readonly BLOCKED = TaskStatus.create(TaskStatusEnum.BLOCKED);
    public static readonly IN_REVIEW = TaskStatus.create(TaskStatusEnum.IN_REVIEW);
    public static readonly REJECTED = TaskStatus.create(TaskStatusEnum.REJECTED);
    public static readonly DRAFT = TaskStatus.create(TaskStatusEnum.DRAFT);
    public static readonly READY = TaskStatus.create(TaskStatusEnum.READY);

    // Static Factory Methods (lowercase) for Aggregate usage
    public static todo(): TaskStatus { return TaskStatus.TODO; }
    public static inProgress(): TaskStatus { return TaskStatus.IN_PROGRESS; }
    public static inQc(): TaskStatus { return TaskStatus.IN_QC; }
    public static qcFailed(): TaskStatus { return TaskStatus.QC_FAILED; }
    public static inAcceptance(): TaskStatus { return TaskStatus.IN_ACCEPTANCE; }
    public static accepted(): TaskStatus { return TaskStatus.ACCEPTED; }
    public static completed(): TaskStatus { return TaskStatus.COMPLETED; }
    public static blocked(): TaskStatus { return TaskStatus.BLOCKED; }
    public static inReview(): TaskStatus { return TaskStatus.IN_REVIEW; }
    public static rejected(): TaskStatus { return TaskStatus.REJECTED; }
    public static draft(): TaskStatus { return TaskStatus.DRAFT; }
    public static ready(): TaskStatus { return TaskStatus.READY; }

    public getValue(): TaskStatusEnum {
        return this.value;
    }

    public isCompleted(): boolean {
        return this.value === TaskStatusEnum.COMPLETED;
    }

    public equals(other: TaskStatus): boolean {
        return this.value === other.value;
    }
    
    public toString(): string {
        return this.value;
    }
}
"""

with open(task_status_path, 'w', encoding='utf-8') as f:
    f.write(new_content)

print("Updated TaskStatus VO")


In [ ]:
task_priority_path = r'src/app/modules/tasks/domain/value-objects/task-priority.vo.ts'

new_priority_content = """/**
 * TaskPriority Value Object
 * 
 * Layer: Domain
 * DDD Pattern: Value Object
 */
export enum TaskPriorityEnum {
    LOW = 'LOW',
    MEDIUM = 'MEDIUM',
    HIGH = 'HIGH',
    CRITICAL = 'CRITICAL'
}

export class TaskPriority {
    private readonly value: TaskPriorityEnum;
    private static readonly cache = new Map<string, TaskPriority>();

    private constructor(value: TaskPriorityEnum) {
        this.value = value;
    }

    public static create(value: string | TaskPriorityEnum): TaskPriority {
        const strValue = value as string;
        if (TaskPriority.cache.has(strValue)) {
            return TaskPriority.cache.get(strValue)!;
        }

        const validValues = Object.values(TaskPriorityEnum) as string[];
        if (validValues.includes(strValue)) {
            const instance = new TaskPriority(strValue as TaskPriorityEnum);
            TaskPriority.cache.set(strValue, instance);
            return instance;
        }
        throw new Error(`Invalid TaskPriority: ${value}`);
    }

    // Static Instances
    public static readonly LOW = TaskPriority.create(TaskPriorityEnum.LOW);
    public static readonly MEDIUM = TaskPriority.create(TaskPriorityEnum.MEDIUM);
    public static readonly HIGH = TaskPriority.create(TaskPriorityEnum.HIGH);
    public static readonly CRITICAL = TaskPriority.create(TaskPriorityEnum.CRITICAL);

    // Static Factory Methods (lowercase)
    public static low(): TaskPriority { return TaskPriority.LOW; }
    public static medium(): TaskPriority { return TaskPriority.MEDIUM; }
    public static high(): TaskPriority { return TaskPriority.HIGH; }
    public static critical(): TaskPriority { return TaskPriority.CRITICAL; }

    public getValue(): TaskPriorityEnum {
        return this.value;
    }

    public equals(other: TaskPriority): boolean {
        return this.value === other.value;
    }
    
    public toString(): string {
        return this.value;
    }
}
"""

with open(task_priority_path, 'w', encoding='utf-8') as f:
    f.write(new_priority_content)

print("Updated TaskPriority VO")


In [ ]:
# Fix TaskAggregate and add createTask factory

task_agg_path = "src/app/modules/tasks/domain/aggregates/task.aggregate.ts"

content = """import { AggregateRoot } from '@domain/base/aggregate-root';
import { TaskId } from '@tasks/domain/value-objects/task-id.vo';
import { WorkspaceId } from '@domain/value-objects/workspace-id.vo';
import { UserId } from '@domain/value-objects/user-id.vo';
import { TaskStatus } from '@tasks/domain/value-objects/task-status.vo';
import { TaskPriority } from '@tasks/domain/value-objects/task-priority.vo';
import { Money } from '@tasks/domain/value-objects/money.vo';
import { Progress } from '@tasks/domain/value-objects/progress.vo';
import { Subtask } from '@tasks/domain/entities/subtask.entity';
import { TaskDependency } from '@tasks/domain/entities/task-dependency.entity';

export interface TaskProps {
    workspaceId: WorkspaceId;
    title: string;
    description: string;
    status: TaskStatus;
    priority: TaskPriority;
    assigneeId: UserId | null;
    estimatedCost: Money | null;
    progress: Progress;
}

/**
 * Task Aggregate Root
 */
export class TaskAggregate extends AggregateRoot<TaskId> {
    private _subtasks: Subtask[] = [];
    private _dependencies: TaskDependency[] = [];

    public readonly workspaceId: WorkspaceId;
    public title: string;
    public description: string;
    public status: TaskStatus;
    public priority: TaskPriority;
    public assigneeId: UserId | null;
    public estimatedCost: Money | null;
    public progress: Progress;

    private constructor(
        id: TaskId,
        props: TaskProps
    ) {
        super(id);
        this.workspaceId = props.workspaceId;
        this.title = props.title;
        this.description = props.description;
        this.status = props.status;
        this.priority = props.priority;
        this.assigneeId = props.assigneeId;
        this.estimatedCost = props.estimatedCost;
        this.progress = props.progress;
    }

    public static create(
        id: TaskId,
        workspaceId: WorkspaceId,
        title: string
    ): TaskAggregate {
        const props: TaskProps = {
            workspaceId,
            title,
            description: '',
            status: TaskStatus.todo(),
            priority: TaskPriority.medium(),
            assigneeId: null,
            estimatedCost: null,
            progress: Progress.zero()
        };
        return new TaskAggregate(id, props);
    }
    
    public static restore(
        id: TaskId,
        props: TaskProps
    ): TaskAggregate {
        return new TaskAggregate(id, props);
    }

    public addSubtask(subtask: Subtask): void {
        this._subtasks.push(subtask);
    }

    public get subtasks(): ReadonlyArray<Subtask> {
        return [...this._subtasks];
    }
}

// Factory function
export function createTask(
    id: TaskId,
    workspaceId: WorkspaceId,
    title: string
): TaskAggregate {
    return TaskAggregate.create(id, workspaceId, title);
}
"""

with open(task_agg_path, "w", encoding="utf-8") as f:
    f.write(content)
print(f"Updated {task_agg_path} with fix.")